In [1]:
from sagemaker import get_execution_role
import sagemaker
import boto3
from datetime import date

session = boto3.session.Session()
sagemaker_session = sagemaker.session.Session(boto_session=session)
# #same function
# sess = sage.Session()

role = 'arn:aws:iam::002224604296:role/service-role/AmazonSageMaker-ExecutionRole-20200402T124851'
aws_region = session.region_name
print(f'AWS region:{aws_region}')

AWS region:us-east-1


In [3]:
model_path = "s3://{}/output".format(sagemaker.Session().default_bucket())
s3_model_url = model_path+'/sagemaker-recsys-graph-2020-11-17-03-15-32-694/'
# #same function：
# model_path = "s3://{}/output".format(sagemaker_session.default_bucket())
print(model_path)

s3://sagemaker-us-east-1-002224604296/output


In [29]:
dt = datetime.now()
dd = "{}-{}-{}-{}-{}-{}".format(dt.year,dt.month,dt.day,dt.hour,dt.minute,dt.second)

In [30]:
TrainJobName = 'recsys-graph-train-{}'.format(dd)
training_image = '002224604296.dkr.ecr.us-east-1.amazonaws.com/sagemaker-recsys-graph-train'
training_instance_type = 'ml.p2.xlarge'
training_params = {
    "RoleArn": role,
    "TrainingJobName": TrainJobName,
    "AlgorithmSpecification": {
        "TrainingImage": training_image,
        "TrainingInputMode": "File"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "{}".format(training_instance_type),
        "VolumeSizeInGB": 1
    },
    "InputDataConfig": [
        {
            "ChannelName": "training",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "{}".format(s3_model_url),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }
    ],
    "OutputDataConfig": {
        "S3OutputPath": "{}".format(s3_model_url)
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60
    }
}

In [31]:
%%time

# sm_session = Session()
sm = boto3.client('sagemaker')
sm.create_training_job(**training_params)

CPU times: user 20.5 ms, sys: 0 ns, total: 20.5 ms
Wall time: 230 ms


{'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:002224604296:training-job/recsys-graph-train-2020-11-25-7-4-48',
 'ResponseMetadata': {'RequestId': '2f5ca844-5e8e-4368-ac52-2e9fc385bff2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2f5ca844-5e8e-4368-ac52-2e9fc385bff2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '111',
   'date': 'Wed, 25 Nov 2020 07:04:52 GMT'},
  'RetryAttempts': 0}}

In [33]:
status = sm.describe_training_job(TrainingJobName=TrainJobName)['TrainingJobStatus']
print(status)

InProgress
